In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

examples = [
    {
        "input": "Top Gun",
        "output": "🛩️👨‍✈️🔥"
    },
    {
        "input": "The Godfather",
        "output": "👨‍👨‍👦🔫🍝"
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative assistant that converts movie titles into exactly three emojis. "
               "Always respond with exactly 3 emojis that best represent the movie. "
               "Do not include any text, just the three emojis."),
    few_shot_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def ask_movie(movie_title):
    response = chain.invoke({"input": movie_title,})
    
    memory.save_context(
        {"input": movie_title},
        {"output": response.content}
    )

    print(f'"{movie_title}" -> "{response.content}"')

In [25]:
ask_movie("The Matrix")

"The Matrix" -> "🕶️💊💥"


In [26]:
ask_movie("Finding Nemo")

"Finding Nemo" -> "🐠🔍🌊"


In [27]:
ask_movie("Squid game")

"Squid game" -> "🦑🎮💰"


In [28]:
ask_movie("Parasite")

"Parasite" -> "🏠👨‍👩‍👧‍👦💰"


In [29]:
ask_movie("Titanic")

"Titanic" -> "🚢❄️💔"


In [30]:
ask_movie("Harry Potter")

"Harry Potter" -> "⚡🧙‍♂️🔮"


In [31]:
ask_movie("Jurassic Park")

"Jurassic Park" -> "🦖🌴🚗"
